### Import Packages

In [ ]:
from animate import animate_trial
import json
from biomechanics import *
from helper import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

### Animate Trial

In [ ]:
# animate_trial(trial=1, energy=False, efficiency=True, azim=320, player_color='green')

### AVERAGE TOTAL ENERGY of various body parts across all trials

In [ ]:
# Plots the average total kinetic energy of various body parts across all trials

# Align tracking data for consistency across trials
aligned_data = align_tracking_data(data)

# Define the body parts and corresponding joint pairs
selected = {
    'upper_arm': [('R_SHOULDER', 'R_ELBOW'), ('L_SHOULDER', 'L_ELBOW')],
    'forearm': [('R_ELBOW', 'R_WRIST'), ('L_ELBOW', 'L_WRIST')],
    'thigh': [('R_HIP', 'R_KNEE'), ('L_HIP', 'L_KNEE')],
    'lower_leg': [('R_KNEE', 'R_ANKLE'), ('L_KNEE', 'L_ANKLE')],
}

# Dictionary to store total energies for each body part across all frames
total_energies_per_frame = {key: [] for key in selected.keys()}

# Iterate through each trial
for trial in range(len(aligned_data)):
    num_frames = len(aligned_data[trial]['tracking']) # Number of frames in the trial
    trial_energies = {key: [] for key in selected.keys()} # Store kinetic energy for each body part for each frame

    # Iterate through each body part and calculate the kinetic energy for each frame
    for body_part, joints_list in selected.items():
        for joints in joints_list:
            for frame in range(num_frames-1):
                # Calculate total kinetic energy for the current body part and frame
                trial_energies[body_part].append(kinetic_energy_total(body_part, joints[0], joints[1], frame, trial))

    # Append the energy values for this trial to the total energies dictionary
    for body_part in selected.keys():
        total_energies_per_frame[body_part].append(trial_energies[body_part])
        
# Calculate the average kinetic energy for each body part across all trials
average_energies = {key: np.mean(total_energies_per_frame[key], axis=0) for key in total_energies_per_frame.keys()}

# Create the plot
fig, ax = plt.subplots(figsize = (8, 8))
colors = plt.cm.get_cmap('tab10', len(selected.keys())) # Use colormap for different body parts

# Plot the average kinetic energy for each body part
for i, body_part in enumerate(average_energies.keys()):
    ax.plot(range(len(average_energies[body_part])), average_energies[body_part], color=colors(i), label=body_part, linestyle='-')

# Add vertical line to find when the ball is released (frame 30)
ax.axvline(30, color='black', linestyle='--')
ax.text(32, 1, f'Ball is\nReleased', color='black', verticalalignment='bottom')

# Add plot legend, labels, and title
plt.legend()
plt.xlabel('Frame')
plt.ylabel('Kinetic Energy (J)')
plt.title('Average Kinetic Energy over All Free Throws')
plt.xlim(0,45)
plt.show()

### RELATIVE ENERGY CONTRIBUTIONS from the right side of the body for all free throw trials

In [ ]:
# Plots the relative energy contributions from the right side of the body for all free throw trials, with markers indicating whether the shot was made or missed.

# Create a new figure and axis for plotting
fig, ax = plt.subplots(figsize = (12, 6))

rel_eng = [] # To store relative energy contributions
result = [] # To store results (made or missed)
colors = [] # To store color codes for the points (red for missed, green for made)

# Loop through each trial
for trial in range(125):
    # Calculate the relative energy contribution for the current trial
    rel_eng.append(relative_energy_contribution(trial))
    # Retrieve the result of the trial (whether the shot was made or missed)
    result.append(data[trial]['result'])

    # Assign colors based on the trial outcome (red for missed, green for made)
    if result[-1] == 'missed':
        colors.append('red')
    elif result[-1] == 'made':
        colors.append('green')

# Create a scatter plot of the relative energy contributions with colors based on results
ax.scatter((range(125)), rel_eng, color = colors)

# Create custom legend elements for 'made' and 'missed' shots
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Made',
                           markerfacecolor='green', markersize=10),
                   Line2D([0], [0], marker='o', color='w', label='Missed',
                           markerfacecolor='red', markersize=10)]

# Add plot legend, labels, and title
ax.legend(handles=legend_elements)
plt.xlabel('Trial')
plt.ylabel('Energy Contribution from the Right Side of the Body (%)')
plt.title('Percent Energy Contribution from the RIGHT Side of the Body')
plt.show()

### AVERAGE KINETIC ENERGY EFFICIENCY of a given joint across all free throw trials

In [ ]:
# Plots the average kinetic energy efficiency of a given joint across all free throw trials, with markers indicating whether the shot was made or missed.

# Set joint
joint = 'R_KNEE'

# Create a new figure and axis for plotting
fig, ax = plt.subplots(figsize = (12, 6))

avg_eff = [] # To store average kinetic energy efficiencies
result = [] # To store results (made or missed)
colors = [] # To store color codes for the points (red for missed, green for made)

# Loop through each trial
for trial in range(125):
    # Calculate the average energy efficiency for the given joint in the current trial
    avg_eff.append(average_kinetic_energy_efficiency(joint, trial))
    # Retrieve the result of the trial (whether the shot was made or missed)
    result.append(data[trial]['result'])

    # Assign colors based on the trial outcome (red for missed, green for made)
    if result[-1] == 'missed':
        colors.append('red')
    elif result[-1] == 'made':
        colors.append('green')

# Create a scatter plot of the average energy efficiencies with colors based on results
ax.scatter((range(125)), avg_eff, color = colors)

# Create custom legend elements for 'made' and 'missed' shots
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Made',
                           markerfacecolor='green', markersize=10),
                   Line2D([0], [0], marker='o', color='w', label='Missed',
                           markerfacecolor='red', markersize=10)]

# Add plot legend, labels, and title
ax.legend(handles=legend_elements)
plt.xlabel('Trial')
plt.ylabel('Kinetic Energy Efficiency (After/Before)')
plt.title(f'Average Energy Transfer Efficiency for the {joint}')
plt.show()

### KINETIC ENERGY TRANSFER EFFICIENCY of given joints over the duration of a free throw trial

In [ ]:
# Plots the kinetic energy transfer efficiency of selected joints over the duration of a free throw trial

# Set trial
trial = 1

# Select joints to display
selected = ['R_ELBOW', 'L_ELBOW']
# selected = ['R_ELBOW', 'L_ELBOW', 'R_SHOULDER', 'L_SHOULDER', 'R_KNEE', 'L_KNEE', 'R_HIP', 'L_HIP']

# Create a color map for the joints
colors = plt.cm.get_cmap('tab10', len(selected))

# Create a new figure and axis for plotting
fig, ax = plt.subplots(figsize = (8, 8))

# Define frame range
start = 0
end = 43

# Loop through each selected joint
for i, joint in enumerate(selected):
    eff = [] # List to store energy efficiency per frame

    # Loop through each frame from start to end
    for frame in range(start, end):
        # Calculate the kinetic energy efficiency for the joint at the current frame
        eff.append(kinetic_energy_efficiency_norm(joint, frame, trial) * 100)
    
    # Scatter plot the energy efficiency for the joint across frames
    ax.scatter((range(start, end)), eff, color = colors(i), label = f'{joint}')

# Add vertical line to find when the ball is released (at frame 30)
ax.axvline(30, color='black', linestyle='--')
ax.text(31, 75, f'Ball is\nReleased', color='black', verticalalignment='bottom')

# Add legend, labels, and title to the plot
plt.legend()
plt.xlabel('Frame')
plt.ylabel('Kinetic Energy Efficiency (%)')
plt.title('Kinetic Energy Transfer Efficiency')
plt.xlim(0, 45)
plt.show()

In [ ]:
# Plot to show TOTAL KINETIC ENERGIES of body parts over the length of the freethrow
trial = 1

selected = {('R_SHOULDER', 'R_ELBOW'):'upper_arm',
            ('L_SHOULDER', 'L_ELBOW'): 'upper_arm',
            ('R_ELBOW', 'R_WRIST'): 'forearm',
            ('L_ELBOW', 'L_WRIST'): 'forearm',

            ('R_HIP', 'R_KNEE'): 'thigh',
            ('L_HIP', 'L_KNEE'): 'thigh',
            ('R_KNEE', 'R_ANKLE'): 'lower_leg',
            ('L_KNEE', 'L_ANKLE'): 'lower_leg',}

colors = plt.cm.get_cmap('tab10', len(selected.keys()))

fig, ax = plt.subplots(figsize = (8, 8))

for i, joints in enumerate(selected.keys()):
    energ = []
    for frame in range(0, 43):
        energ.append(kinetic_energy_total(selected[joints], joints[0], joints[1], frame, trial))
    ax.scatter((range(0,43)), energ, color = colors(i), label = f'{joints[0][0]}_{selected[joints]}')


# # Plot vertical lines to indicate phases of movement
# for loc, label in zip([165, 170, 175], ['Load', 'Release', 'Follow \nThrough']):
#     ax.axvline(loc, color='black', linestyle='--')
#     # Optionally, annotate the plot with the value of the maximum energy
#     ax.text(loc + 1, 4.5, f'{label}', color='black', verticalalignment='bottom')

# Add vertical line to find when the ball is released
ax.axvline(30, color='black', linestyle='--')
ax.text(32, 3, f'Ball is\nReleased', color='black', verticalalignment='bottom')

plt.legend()
plt.xlabel('Frame')
plt.ylabel('Kinetic Energy (J)')
plt.title(f'Total Kinetic Energy over the course of Trial #{trial}')
plt.xlim(0, 45)
plt.show()

In [ ]:
# Plot to show DISPLACEMENT of joints over the length of the freethrow
trial = 1

sequence = {('R_SHOULDER', 'R_ELBOW'):'upper_arm',
            ('L_SHOULDER', 'L_ELBOW'): 'upper_arm',
            ('R_ELBOW', 'R_WRIST'): 'forearm',
            ('L_ELBOW', 'L_WRIST'): 'forearm',

            ('R_HIP', 'R_KNEE'): 'thigh',
            ('L_HIP', 'L_KNEE'): 'thigh',
            ('R_KNEE', 'R_ANKLE'): 'lower_leg',
            ('L_KNEE', 'L_ANKLE'): 'lower_leg',

            ('R_SHOULDER', 'R_HIP'): 'trunk_vertical',
            ('L_SHOULDER', 'L_HIP'): 'trunk_vertical',
            ('R_SHOULDER', 'L_SHOULDER'): 'trunk_horizontal',
            ('R_HIP', 'L_HIP'): 'trunk_horizontal',

            }

fig, ax = plt.subplots(3, 1, figsize = (12, 8))

colors = plt.cm.get_cmap('tab20', len(sequence.keys()))

ylabels = ['X-Displacement (m)', 'Y-Displacement (m)', 'Z-Displacement (m)']

for d in range(3):
    for i, joints in enumerate(sequence.keys()):
        rvel = []
        for frame in range(0, 43):
            rvel.append(distance(joints[0], joints[1], frame, trial)[d])
        ax[d].scatter((range(0,43)), rvel, color=colors(i), label = joints)
    ax[d].set_ylabel(ylabels[d])

plt.legend()
plt.suptitle(f'Joint Displacement Over the Length of Trial #{trial}')
plt.xlabel('Frame')
plt.show()

In [ ]:
# Plot to show ROTATIONAL KINETIC ENERGIES of body parts over the length of the freethrow
sequence = {('R_SHOULDER', 'R_ELBOW'):'upper_arm',
            ('L_SHOULDER', 'L_ELBOW'): 'upper_arm',
            ('R_ELBOW', 'R_WRIST'): 'forearm',
            ('L_ELBOW', 'L_WRIST'): 'forearm',

            ('R_HIP', 'R_KNEE'): 'thigh',
            ('L_HIP', 'L_KNEE'): 'thigh',
            ('R_KNEE', 'R_ANKLE'): 'lower_leg',
            ('L_KNEE', 'L_ANKLE'): 'lower_leg',

            ('R_SHOULDER', 'R_HIP'): 'trunk_vertical',
            ('L_SHOULDER', 'L_HIP'): 'trunk_vertical',
            }

trial = 1

fig, ax = plt.subplots(figsize = (8, 8))

colors = plt.cm.get_cmap('tab10', len(sequence.keys()))

for i, joints in enumerate(sequence.keys()):
    energ = []
    for frame in range(0,43):
        energ.append(kinetic_energy_rot(sequence[joints], joints[0], joints[1], frame, trial))
    ax.scatter(range(0,43), energ, color=colors(i), label = joints[0][0] + '_' + sequence[joints])

plt.legend()
plt.xlabel('Frame')
plt.ylabel('Kinetic Energy (J)')
plt.title(f'Rotational Kinetic Energy over Trial #{trial}')
plt.show()

In [ ]:
# Plot to show TRANSLATIONAL KINETIC ENERGIES of body parts over the length of the freethrow
sequence = {('R_SHOULDER', 'R_ELBOW'):'upper_arm',
            ('L_SHOULDER', 'L_ELBOW'): 'upper_arm',
            ('R_ELBOW', 'R_WRIST'): 'forearm',
            ('L_ELBOW', 'L_WRIST'): 'forearm',

            ('R_HIP', 'R_KNEE'): 'thigh',
            ('L_HIP', 'L_KNEE'): 'thigh',
            ('R_KNEE', 'R_ANKLE'): 'lower_leg',
            ('L_KNEE', 'L_ANKLE'): 'lower_leg',

            ('R_SHOULDER', 'R_HIP'): 'trunk_vertical',
            ('L_SHOULDER', 'L_HIP'): 'trunk_vertical',
            }

trial = 1

fig, ax = plt.subplots(figsize = (8, 8))

colors = plt.cm.get_cmap('tab10', len(sequence.keys()))

for i, joints in enumerate(sequence.keys()):
    energ = []
    for frame in range(0,43):
        energ.append(kinetic_energy_trans(sequence[joints], joints[0], joints[1], frame, trial))
    ax.scatter((range(0,43)), energ, color=colors(i), label = joints[0][0] + '_' + sequence[joints])

plt.legend()
plt.xlabel('Frame')
plt.ylabel('Kinetic Energy (J)')
plt.title(f'Rotational Kinetic Energy over Trial #{trial}')
plt.show()

In [ ]:
# # Get the peak kinetic energy of the thigh of each trial

# peak_frames = []

# for trial in range(1, 125):
#     r_thigh = []
#     l_thigh = []
#     for frame in range(130, ball_leaves_hands(trial)):
#         r_thigh.append(kinetic_energy_total('thigh', 'R_HIP', 'R_KNEE', frame, trial))
#         l_thigh.append(kinetic_energy_total('thigh', 'L_HIP', 'L_KNEE', frame, trial))
#     ind = int((r_thigh.index(max(r_thigh)) + l_thigh.index(max(l_thigh))) / 2)
#     peak_frames.append(130 + ind)
# min(peak_frames), max(peak_frames)